# Theme Park Django PWA Workbook

This notebook teaches the design, structure, and deployment of the **Theme Park Progressive Web App (PWA)** built with Django.

We will cover:
1. Purpose and design logic  
2. Directory tree and file explanations  
3. Steps to create the Django PWA  
4. Security-first architecture  
5. Real-time features with Django Channels  
6. PWA service worker and offline behaviour  
7. Deployment and CI/CD


## 1. Introduction & Purpose

### What is a Progressive Web Application (PWA)?

### Why are we choosing Django for this PWA?

### What is the Theme Park Scenario?
We are designing and building a web‑based information system for a Theme Park, which will be device‑responsive and support offline behaviour.


# Theme Park Django PWA Workbook
This notebook teaches the design, structure, and deployment of the **Theme Park Progressive Web App (PWA)** built with Django.


### 🎯 Activity 1: Introduction
- In your own words, explain what a PWA is.
- Why are we using Django for this project?

✅ Success criteria: mentions offline/installable features, Django’s batteries‑included architecture, Theme Park scenario relevance.


## 2. Purpose and Design Logic

The Theme Park Django PWA is designed to:
- Provide a **secure, classroom‑ready web app** using Django.
- Demonstrate **modern web features**: offline support, service workers, manifest.
- Teach **relational database design** using the Django ORM.
- Showcase **authentication and authorisation**: Django auth, roles, permissions.
- Model **DevOps practices**: migrations, testing, CI/CD, containerization.


## 3. Project Structure (Directory Tree) and File Extensions


In [ ]:
# This gives you a mental map of the Django project before diving into the code.

print("""themepark_django/
├── pyproject.toml              # uv-based dependency management
├── uv.lock                     # lockfile for reproducibility
├── manage.py                   # Django management entry point
├── themepark/                  # Project settings package
│   ├── settings.py             # Security-first settings
│   ├── urls.py                 # Root URL routing
│   ├── asgi.py                 # ASGI entrypoint (for Channels)
│   └── wsgi.py                 # WSGI entrypoint
├── customers/                  # App: customers
│   ├── models.py
│   ├── views.py
│   ├── urls.py
│   ├── admin.py
│   └── tests.py
├── bookings/                   # App: bookings
│   ├── models.py
│   ├── views.py
│   ├── urls.py
│   ├── admin.py
│   └── tests.py
├── templates/                  # Django templates
│   ├── base.html
│   ├── index.html
│   ├── offline.html
│   └── ...
├── static/                     # CSS, JS, images
│   ├── css/
│   ├── js/service-worker.js
│   ├── js/app.js
│   └── manifest.webmanifest
├── workbooks/
│   └── ThemePark_Django_PWA_Workbook.ipynb
└── .github/workflows/
    ├── ci.yml
    └── codeql.yml
""")


### 🎯 Activity 2: Project Structure
- Pick three files from the directory tree and describe their role.

✅ Success criteria: correct file purpose, explains interactions, uses correct terminology.


## 4. Environment Setup — Steps to Create your Django PWA

### Virtual Environments (uv)

### pyproject.toml


In the cell below, `!uv` means "run uv as a shell command inside Jupyter Notebook".


In [ ]:
# Install all dependencies directly from the notebook (requires uv installed in the environment)
!uv sync


### 🎯 Activity 3: Environment Setup
- Run the install command.
- Which tools in `pyproject.toml` are for security?

✅ Success criteria: identifies Bandit, Semgrep, Django security middleware, explains dev vs prod differences.


## 5. Django App Basics


In [ ]:
# Create a new Django project (only run once in a clean directory)
# !uv run django-admin startproject themepark .
print("Django project would be created with: uv run django-admin startproject themepark .")


### 🎯 Activity 4: Django Basics
- What does `django-admin startproject` do?

✅ Success criteria: creates settings, URL routing, WSGI/ASGI entry points.


## 6. Database Integration — Django ORM

### Why ORMs matter
- Abstract away raw SQL.
- Reduce risk of SQL injection.
- Make schema changes reproducible via migrations.


In [ ]:
# Example: Creating the customers app (run once)
# !uv run python manage.py startapp customers
print("Django app 'customers' would be created with: uv run python manage.py startapp customers")


### 🎯 Activity 5: Database Integration
- Why use Django ORM instead of raw SQL?

✅ Success criteria: ORM abstraction, security via parameterization, reproducibility.


## 7. Models & Constraints

We will model **customers** and **bookings** as Django models.


In [ ]:
from django.db import models

class Customer(models.Model):
    first_name = models.CharField(max_length=100)
    last_name  = models.CharField(max_length=100)
    email      = models.EmailField(unique=True)
    phone      = models.CharField(max_length=20, blank=True)

    def __str__(self):
        return f"{self.first_name} {self.last_name}"

class Booking(models.Model):
    customer       = models.ForeignKey(Customer, on_delete=models.CASCADE, related_name="bookings")
    booking_date   = models.DateField()
    number_of_guests = models.PositiveIntegerField()
    status         = models.CharField(max_length=20, default="Pending")

    class Meta:
        constraints = [
            models.CheckConstraint(
                check=models.Q(number_of_guests__lte=10),
                name="booking_max_10_guests",
            )
        ]

    def __str__(self):
        return f"Booking #{self.id} for {self.customer} on {self.booking_date}"


### 🎯 Activity 6: Models & Constraints
- Why is the `booking_max_10_guests` constraint important?

✅ Success criteria: explains business logic enforcement, prevents abuse, supports data integrity.


## 8. Data Dictionary (Access‑Derived)

Below is a **sample data dictionary** for the Theme Park database. Replace or extend this with your full Access‑derived schema.

| Table        | Field            | Data Type      | Description                         |
|-------------|------------------|----------------|-------------------------------------|
| tblCustomers| CustomerID (PK)  | Integer        | Unique customer identifier          |
|             | FirstName        | Text (100)     | Customer first name                 |
|             | LastName         | Text (100)     | Customer last name                  |
|             | Email            | Text (255)     | Unique email address                |
|             | Phone            | Text (20)      | Contact phone number                |
| tblBookings | BookingID (PK)   | Integer        | Unique booking identifier           |
|             | BookingDate      | Date           | Date of the booking                 |
|             | CustomerID (FK)  | Integer        | Links to tblCustomers.CustomerID    |
|             | NumberOfGuests   | Integer        | Number of guests (max 10)          |
|             | BookingStatus    | Text (20)      | e.g. Pending, Confirmed, Cancelled  |


## 9. Mermaid ERD (Theme Park Schema)

The diagram below visualises the relationship between **customers** and **bookings**.

```mermaid
erDiagram
    tblCustomers {
        int CustomerID PK
        string FirstName
        string LastName
        string Email
        string Phone
    }

    tblBookings {
        int BookingID PK
        date BookingDate
        int CustomerID FK
        int NumberOfGuests
        string BookingStatus
    }

    tblCustomers ||--o{ tblBookings : "makes"
```

### 🎯 Activity: ERD
- Add at least one more entity (e.g. `tblRides`) to the ERD.
- Show how it relates to `tblBookings`.


## 10. Templates & Static Assets

- `base.html` → layout with navbar, footer.
- `index.html` → home page.
- `offline.html` → offline fallback page.
- `manifest.webmanifest` → PWA metadata.
- `service-worker.js` → offline caching and asset management.


### 🎯 Activity: Templates & Assets
- How do `manifest.webmanifest` and `service-worker.js` make this app a PWA?

✅ Success criteria: manifest metadata, offline caching, installability.


## 11. PWA Service Worker (Django Staticfiles)

Below is a **sample service worker** tailored for Django staticfiles. It assumes your static files are served under `/static/`.


In [ ]:
service_worker_js = r"""const CACHE_NAME = 'themepark-django-pwa-v1';
const URLS_TO_CACHE = [
  '/',
  '/offline/',
  '/static/css/style.css',
  '/static/js/app.js',
  '/static/img/logo.png'
];

self.addEventListener('install', event => {
  event.waitUntil(
    caches.open(CACHE_NAME).then(cache => {
      return cache.addAll(URLS_TO_CACHE);
    })
  );
});

self.addEventListener('activate', event => {
  event.waitUntil(
    caches.keys().then(cacheNames => {
      return Promise.all(
        cacheNames.map(cacheName => {
          if (cacheName !== CACHE_NAME) {
            return caches.delete(cacheName);
          }
        })
      );
    })
  );
});

self.addEventListener('fetch', event => {
  event.respondWith(
    caches.match(event.request).then(response => {
      return response || fetch(event.request).catch(() => {
        if (event.request.mode === 'navigate') {
          return caches.match('/offline/');
        }
      });
    })
  );
});
"""

print(service_worker_js[:400] + "...\n\n(Truncated for display; save this to static/js/service-worker.js in your project.)")


### 🎯 Activity: Service Worker
- Where should this file live in your Django project?
- How does it interact with `offline.html`?


## 12. Authentication & Authorization

Django provides a built‑in **User** model and authentication system.


In [ ]:
from django.contrib.auth.models import User

# Example: checking if a user is staff
def is_staff_user(user: User) -> bool:
    return user.is_staff

print("Django auth provides User, groups, permissions, and login/logout views.")


### 🎯 Activity 8: Authentication
- What’s the difference between Django session auth and JWT API tokens?

✅ Success criteria: session cookies vs stateless tokens, expiry/roles.


## 13. Admin Dashboard

Django admin is built‑in and auto‑generates CRUD interfaces for registered models.


In [ ]:
from django.contrib import admin
from .models import Customer, Booking

admin.site.register(Customer)
admin.site.register(Booking)

print("Customer and Booking models registered in Django admin.")


### 🎯 Activity 9: Admin Dashboard
- Why must admin routes be role‑protected?

✅ Success criteria: mentions least privilege, example of editing user roles, connects to audit safety.


## 14. Django Channels — Real‑Time Features

Django Channels allows us to add **WebSocket** and real‑time features to our Theme Park app (e.g. live queue updates, live booking notifications).


In [ ]:
# Example ASGI setup snippet (conceptual)
asgi_example = r"""# themepark/asgi.py
import os
from django.core.asgi import get_asgi_application
from channels.routing import ProtocolTypeRouter, URLRouter
from channels.auth import AuthMiddlewareStack
import queue.routing

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'themepark.settings')

django_asgi_app = get_asgi_application()

application = ProtocolTypeRouter({
    'http': django_asgi_app,
    'websocket': AuthMiddlewareStack(
        URLRouter(queue.routing.websocket_urlpatterns)
    ),
})
"""

print(asgi_example[:400] + "...\n\n(Truncated for display; this shows how Channels integrates with ASGI.)")


### 🎯 Activity: Real‑Time Features
- Suggest one real‑time feature for the Theme Park (e.g. live ride wait times).
- How could a WebSocket help implement it?


## 15. Testing & Security

We use **unit tests**, **integration tests**, and **SAST/DAST tools** to keep the project secure and reliable.


In [ ]:
# Example pytest-style test (conceptual)
def test_homepage(client):
    response = client.get("/")
    assert response.status_code == 200

print("Example test: homepage returns HTTP 200.")


### 🎯 Activity 10: Security Testing
- Run Bandit (in your repo, not inside this notebook).
- What type of issues does it detect?

✅ Success criteria: identifies issue type (hardcoded secrets, unsafe functions), explains importance.


## 16. Deployment Preparation

We will containerise the Django app and prepare it for deployment.


In [ ]:
%%bash
cat > Dockerfile <<'EOF'
FROM python:3.11-slim
WORKDIR /app
COPY pyproject.toml .
RUN pip install uv && uv sync
COPY . .
CMD ["uv", "run", "gunicorn", "themepark.wsgi:application", "-b", "0.0.0.0:8080"]
EOF

echo "Dockerfile created (in this environment, this is just a demonstration)."


### 🎯 Activity 11: Deployment Prep
- Why use Docker instead of running Django directly?

✅ Success criteria: reproducibility, environment consistency, CI/CD pipelines.


## 17. CI/CD Automation

We use **GitHub Actions** to run tests, linting, and security scans automatically.


### 🎯 Activity 12: CI/CD
- What happens if a test fails in GitHub Actions?

✅ Success criteria: pipeline stops, feedback loop, quality assurance.


## 18. Going Live

When deploying to production, we must:
- Use environment variables for secrets.
- Enable HTTPS (TLS).
- Use a production database (e.g. PostgreSQL).
- Harden security settings (e.g. `SECURE_HSTS_SECONDS`, `SECURE_SSL_REDIRECT`).


### 🎯 Activity 13: Going Live
- Why must secrets be stored in environment variables?

✅ Success criteria: explains risk of hardcoding, mentions rotation/audit, compliance.


## 19. Reflection & Wrap‑Up

### ✅ Reflection
- List three security measures we added and explain their purpose.

✅ Success criteria: identifies measures (CSP, rate limiting, DB constraints, auth), explains clearly, connects to overall posture.
